# Distancias

In [1]:
from configparser import ConfigParser
import json
import csv
import numpy as np
import pandas as pd
import googlemaps
from unidecode import unidecode

## 1. Distance from Port of Las Palmas to final destination

### Coordinates ports

In [2]:
# Dataframe coordinates origin-destination ports
ports = pd.read_csv('../data/raw/CoordenadasPuertos.csv')

# New variable latlon
ports['latlon'] = ports['lat'].astype(str) + ',' + ports['lon'].astype(str)

In [3]:
# Port of Las Palmas
port_lp = ports[ports['port_name'] =='LAS PALMAS']
port_lp_latlon_a = tuple(map(float, port_lp['latlon'].tolist()[0].split(',')))

# Dataframe coordinates destination. Place where books will be finally delivered.
final_dest_latlon_b = (28.086528,-15.434806)

### Google Maps API: Distance from Port of Las Palmas to delivery point

In [4]:
# Get google maps api key
config = ConfigParser()
config.read('../config/mysecrets.cfg')
API_key = config['googlemaps']['auth_key']

# Instantiate google maps client
gmaps = googlemaps.Client(key=API_key)

In [5]:
# Distance with Google Maps API
portlp_final_dist = gmaps.distance_matrix(port_lp_latlon_a, final_dest_latlon_b, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
portlp_final_dist = portlp_final_dist/1000

# Crear dataframe
port_lp = port_lp.drop(columns=['unlocode', 'lat', 'lon'])
port_lp.rename(columns={'latlon': 'port_latlon'}, inplace=True)
port_lp['punto_llegada_latlon'] = str(final_dest_latlon_b).replace('(','').replace(')','').replace(' ', '')
port_lp['descarga_llegada_distancia_km'] = portlp_final_dist

# Write .csv
port_lp.to_csv('../data/processed/puertodescarga_llegada_distance.csv')

## 2. Distance from ports to Port of Las Palmas

In [6]:
# Distances mainland ports to Las Palmas
interport_main = pd.read_csv('../data/raw/CuadroDistancias_PuertosPeninsula.csv')
interport_main.rename(columns={'Unnamed: 0': 'port_name', 'LAS PALMAS': 'distanceLP_nauticalmiles'}, inplace=True)
interport_main = interport_main[['port_name', 'distanceLP_nauticalmiles']]
interport_main = interport_main[interport_main['port_name'].isin(['TENERIFE', 'LAS PALMAS']) == False]

# Distances canarian ports to Las Palmas
interport_island = pd.read_csv('../data/raw/CuadroDistancias_PuertosCanarias.csv')
interport_island = interport_island[['port_name', 'distanceLP_nauticalmiles']]

# Distances ports
interport_dists = pd.concat([interport_main, interport_island], ignore_index=True)

# Rename
interport_dists.rename(columns={'distanceLP_nauticalmiles': 'distanceLP_nm'}, inplace=True)

# From Nautical Miles to Kilometers
# 1 nm = 1.852 km
interport_dists['distanceLP_km'] = interport_dists['distanceLP_nm'] * 1.852

# Write .csv
interport_dists.to_csv('../data/processed/port_portlp_distance.csv')


## 3. Shortest distance to port

### Get data

#### Coordinates cities

In [7]:
# Dataframe coordinates origin-cities
# Opening json file and loading it as a dictionary python object
with open('../data/raw/spain-cities.json', 'r') as f:
    cities = json.load(f)

# Origin data: create .csv
canarias_keep = ['San Sebastián de la Gomera', 'Santa Cruz de la Palma', 'Valverde', 
               'Santa Cruz de Tenerife', 'Las Palmas de Gran Canaria', 'Arrecife', 
               'Puerto del Rosario']
# writer
with open('../data/processed/origins.csv', 'w+') as file: # create file if it doesn't exist
    writer = csv.writer(file, delimiter=',', lineterminator='\n')
    writer.writerow(['id', 'name', 'population', 'lat', 'lon'])
    for i in range(len(cities)):
        name_check = cities[i]['fields']['name']
        if name_check in canarias_keep:
            name = [name_check]
            population = [cities[i]['fields']['population']]
            lat = [cities[i]['fields']['coordinates'][0]]
            lon = [cities[i]['fields']['coordinates'][1]]
            row = [i] + name + population + lat + lon
            writer.writerow(row)
        else:
            population = [cities[i]['fields']['population']]
            if population[0] >= 50000:
                name = [name_check]
                lat = [cities[i]['fields']['coordinates'][0]]
                lon = [cities[i]['fields']['coordinates'][1]]
                row = [i] + name + population + lat + lon
                writer.writerow(row)

# reader
origins = pd.read_csv('../data/processed/origins.csv')

# New variable latlon
origins['latlon'] = origins['lat'].astype(str) + ',' + origins['lon'].astype(str)

# Format city names
origins['name'] = origins['name'].str.upper()
origins['name'] = origins['name'].apply(unidecode)


#### Coordinates ports of origin

In [8]:
# Ports of origin
# Not all ports in the dataset are common starting points to go to Gran Canaria.
common_ports = ['ALGECIRAS', 'VALENCIA', 'BARCELONA', 'BILBAO', 'CASTELLON',
                'CADIZ', 'SEVILLA', 'VILAGARCIA', 'ALICANTE', 'SANTANDER', 'VIGO',
                'TENERIFE', 'ARRECIFE', 'LA PALMA', 'LA ESTACA', 'PUERTO DEL ROSARIO', 'LA GOMERA', 'LAS PALMAS']
ports_og = ports[ports['port_name'].isin(common_ports)]

### Google API shortest distance to port

In [9]:
shortest_distances_to_port = []

for coordinate_a in origins['latlon']:
    city_latlon_a = tuple(map(float, coordinate_a.split(',')))
    distance_to_port = np.nan
    for index, row in ports_og.iterrows():
        port_latlon = row['latlon']
        port_latlon_b = tuple(map(float, port_latlon.split(',')))
        distance_temp = gmaps.distance_matrix(city_latlon_a, port_latlon_b, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
        distance_temp = distance_temp/1000
        if np.isnan(distance_to_port):
            distance_to_port = distance_temp
            port_name = row['port_name']
            result_add = [port_name, port_latlon, distance_to_port]
        else:
            if distance_temp < distance_to_port:
                distance_to_port = distance_temp
                port_name = row['port_name']
                result_add = [port_name, port_latlon, distance_to_port] 
    shortest_distances_to_port.append(result_add)
    shortest_distances_to_port_df = pd.DataFrame(shortest_distances_to_port, columns = ['port_name', 'port_latlon', 'shortest_distance_to_port_km'])

og_port_distance = pd.concat([origins, shortest_distances_to_port_df], axis=1)


# Write .csv
og_port_distance.to_csv('../data/processed/city_port_distance.csv')

# # Read .csv
og_port_distance = pd.read_csv('../data/processed/city_port_distance.csv')
og_port_distance.drop(columns=['Unnamed: 0', 'id', 'lat', 'lon'], inplace=True)


In [10]:
og_port_distance[og_port_distance['port_name']=='LAS PALMAS']

,name,population,latlon,port_name,port_latlon,shortest_distance_to_port_km
117,SAN BARTOLOME DE TIRAJANA,52161,"27.92481,-15.57329",LAS PALMAS,"28.141,-15.41635",64.859
124,LAS PALMAS DE GRAN CANARIA,381847,"28.09973,-15.41343",LAS PALMAS,"28.141,-15.41635",10.411
132,SANTA LUCIA,63637,"27.91174,-15.54071",LAS PALMAS,"28.141,-15.41635",57.685
173,TELDE,100015,"27.99243,-15.41915",LAS PALMAS,"28.141,-15.41635",24.596


## 4. Distance from starting points in Gran Canaria to delivery point

In [14]:
# Dataframe de salidas en Gran Canaria y que irán directamente al punto de destino
cityGC_destination_distance = og_port_distance[og_port_distance['port_name']=='LAS PALMAS'].copy()
cityGC_destination_distance.drop(columns=['port_name', 'port_latlon', 'shortest_distance_to_port_km'], inplace=True)
cityGC_destination_distance['punto_llegada_latlon'] = str(final_dest_latlon_b).replace('(','').replace(')','').replace(' ', '')

In [15]:
origins = cityGC_destination_distance.copy()
distance_to_destination = []

for coordinate_a in origins['latlon']:
    city_latlon_a = tuple(map(float, coordinate_a.split(',')))
    distance_temp = gmaps.distance_matrix(city_latlon_a, final_dest_latlon_b, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    distance = distance_temp/1000
    distance_to_destination.append(distance)

# Añadir distancias al dataframe
cityGC_destination_distance['salida_llegada_distancia_km'] = distance_to_destination

# Write .csv
cityGC_destination_distance.to_csv('../data/processed/cityGC_destination_distance.csv')